### Distance computations


Given two matrices `X` and `Y` we want to compute the distance between every row of `X` and every row of `Y`.

In [2]:
import numpy as np

In [22]:
n_X = 100
n_Y = 200
n_features = 100


X = np.random.random((n_X, n_features))
Y = np.random.random((n_Y, n_features))

What we are want to do (but much faster)

In [23]:
def euclidean(X,Y):
    n_X = X.shape[0] # 200
    n_Y = Y.shape[0] # 500
    dists = np.zeros((n_X, n_Y))
    for i in range(n_X):
        dists[i, :] = np.sum((Y - X[i, :])**2, axis=1)
    return np.sqrt(dists)


In [24]:
euclidean_naive = euclidean(X,Y)

In [25]:
from scipy.spatial.distance import cdist
cdist_euclidean = cdist(X, Y,'euclidean')
np.mean(euclidean_naive - cdist_euclidean)

4.019007349143067e-18

#### Han current solution

This code is present in both hello world and numpy indexer and it is heavily explained here

https://hanxiao.io/2020/09/21/Numpy-Tricks-and-A-Strong-Baseline-for-Vector-Index/



In [26]:
def _get_ones(x, y):
    return np.ones((x, y))

def _ext_A(A):
    nA, dim = A.shape
    A_ext = _get_ones(nA, dim * 3)
    A_ext[:, dim : 2 * dim] = A
    A_ext[:, 2 * dim :] = A ** 2
    return A_ext

def _ext_B(B):
    nB, dim = B.shape
    B_ext = _get_ones(dim * 3, nB)
    B_ext[:dim] = (B ** 2).T
    B_ext[dim : 2 * dim] = -2.0 * B.T
    del B
    return B_ext

def _euclidean(A_ext, B_ext):
    sqdist = A_ext.dot(B_ext).clip(min=0)
    return np.sqrt(sqdist)

def _norm(A):
    return A / np.linalg.norm(A, ord=2, axis=1, keepdims=True)

def _cosine(A_norm_ext, B_norm_ext):
    return A_norm_ext.dot(B_norm_ext).clip(min=0) / 2

In [27]:
X_ext = _ext_A(X)
Y_ext = _ext_B(Y)
dists_han = _euclidean(X_ext, Y_ext)

In [28]:
np.mean(dists_han - cdist_euclidean) 

3.0109248427834245e-17

In [29]:
def han_euclidean(X,Y):
    X_ext = _ext_A(X)
    Y_ext = _ext_B(Y)
    return _euclidean(X_ext, Y_ext)

### Buchaca proposal

In [30]:
print(f'X_n={X.shape[0]}, Y_n={Y.shape[0]}')

# Creates a row vector of shape 200
print(f'\nnp.sum(Y**2, axis=1).shape = {np.sum(Y**2, axis=1).shape}')

# Note that vec[:, np.newaxis] puts vec with a superior ndim
# This creates a column vector of shape 200
print(f'\nnp.sum(X**2, axis=1)[:, np.newaxis] = {np.sum(X**2, axis=1)[:, np.newaxis].shape}')

X_n=100, Y_n=200

np.sum(Y**2, axis=1).shape = (200,)

np.sum(X**2, axis=1)[:, np.newaxis] = (100, 1)


When row_vec + col_vec is computed in numpy the vectors are copied as many times as needed to make an elementwise operation valid.

For example, if a row vector has 10 elements and a column vector has 20 the vector
is 

In [31]:
row_vec = np.random.rand(10)
col_vec = np.random.rand(20,1)
(row_vec + col_vec).shape

(20, 10)

In [32]:
def buch_euclidean(X, Y):
    dists = np.sum(Y**2, axis=1) + np.sum(X**2, axis=1)[:, np.newaxis] -2*np.dot(X, Y.T)
    return dists

In [33]:
def buch_euclidean_squared(X, Y):
    dists = np.sum(Y**2, axis=1) + np.sum(X**2, axis=1)[:, np.newaxis] -2*np.dot(X, Y.T)
    return np.sqrt(dists)

In [34]:
dists_han = han_euclidean(X, Y)
dists_buch = buch_euclidean_squared(X, Y)
np.mean(dists_han - dists_buch) 

1.7494894422043218e-16

In [35]:
%%timeit
dists_han = han_euclidean(X, Y)

309 µs ± 17.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [36]:
%%timeit
dists_buch = buch_euclidean(X, Y)

139 µs ± 4.02 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [37]:
%%timeit
dists_buch = buch_euclidean_squared(X, Y)

159 µs ± 3.49 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


We can also do something similar for the cosine distance

In [38]:
def buch_cosine_similarity(X, Y):
    dists = np.dot(X, Y.T)/np.outer(np.linalg.norm(X, ord=2, axis=1),np.linalg.norm(Y, ord=2, axis=1))
    return dists

def buch_cosine_distance(X, Y):
    dists = 1- np.dot(X, Y.T)/np.outer(np.linalg.norm(X, ord=2, axis=1),np.linalg.norm(Y, ord=2, axis=1))
    return dists

In [39]:
buch_cosine_similarity(X,Y)

array([[0.77177385, 0.73774092, 0.73089505, ..., 0.79648348, 0.7110453 ,
        0.7271093 ],
       [0.75280515, 0.69961514, 0.72725627, ..., 0.7674045 , 0.70421417,
        0.75367441],
       [0.77081983, 0.71682104, 0.74690078, ..., 0.80442634, 0.77912576,
        0.75469284],
       ...,
       [0.75471001, 0.667707  , 0.7192179 , ..., 0.75338295, 0.76200319,
        0.73534841],
       [0.74930071, 0.78665022, 0.77064414, ..., 0.80151832, 0.77284409,
        0.76188978],
       [0.73295346, 0.69427336, 0.700622  , ..., 0.75357708, 0.74543188,
        0.7040662 ]])

In [40]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [41]:
np.mean(cosine_similarity(X,Y) - buch_cosine_similarity(X,Y))

-1.6459056340067945e-17